In [1]:
import warnings
warnings.simplefilter(action='ignore')
from Bio import SeqIO
import numpy as np
from tqdm import tqdm
import seaborn as sns
from matplotlib import pyplot as plt
import tensorflow as tf
import keras.backend as K
import keras
import os
from keras.layers import Dense, Flatten, Conv2D, Conv1D, MaxPooling2D, Embedding, Input, Dropout, Reshape, Activation
from keras.models import Sequential, load_model
from keras.utils import multi_gpu_model 
import matplotlib.pylab as plt
from skimage.transform import resize
from keras.callbacks import EarlyStopping
from sklearn.model_selection import StratifiedKFold
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from tqdm import tnrange
tf.logging.set_verbosity(tf.logging.ERROR)

Using TensorFlow backend.


In [2]:
def transform(data): #transform sequence into one hot encoding
    def one_hot_encode(seq):
        mapping = dict(zip("acgnt", range(5)))     
        seq2 = [mapping[i] if i in ['a', 't', 'c', 'g', 'n'] else mapping['n'] for i in seq]
        return np.eye(5)[seq2]
    read_size = 150
    seq = one_hot_encode(data[0])
    seq = np.expand_dims(seq, axis = -1)
    return seq

def read_data(root_dir, file):
    viral = []
    hum = []
    file_path = os.path.join(root_dir, file)
    for seq_record in SeqIO.parse(file_path, "fasta"):
        if('v' in seq_record.id):
            viral.append(seq_record.seq)
        if('chr1' in seq_record.id):
            hum.append(seq_record.seq)
    return viral, hum

In [3]:
root_dir = '/home/ecvol/data/viral/'
read_length=150

### Get training reads from viral sequences

Get 5000 reads from each viral reference

In [25]:
train_seq= []
for idx, seq in enumerate(SeqIO.parse(root_dir + "hpv.fas", "fasta")):
    sequence = str(seq.seq)
    if(not sequence.islower()):
        seq = sequence.lower()
    train_seq.append(sequence)

############################################################################################
smallest_len = min(len(i) for i in train_seq)
num_reads = 5000
rand_reads = np.random.choice(range(smallest_len-152), size = num_reads, replace=False)
train_set = np.empty([len(train_seq)*num_reads,read_length, 5, 1], np.int8)
train_id = 0
for idx, seq in tqdm(enumerate(train_seq), total = len(train_seq)):
    for read_start in rand_reads:
        read = np.array(seq[read_start:read_start+read_length])
        train_set[train_id] = transform(np.expand_dims(read, axis=0))
        train_id += 1
############################################################################################
np.save(root_dir + 'v_ref_reads', train_set) #when loading, call np.random.shuffle()

100%|██████████| 337/337 [00:58<00:00,  5.75it/s]


In [31]:
#print(read, train_set[-1]) #check
print(train_set.shape)

(1685000, 150, 5, 1)


### Get training and test reads from chr1

In [40]:
for idx, seq in enumerate(SeqIO.parse(root_dir+"hg19full.fa", "fasta")):
    train_seq = str(seq.seq)
    if(not train_seq.islower()):
        train_seq = train_seq.lower()
    break

############################################################################################
num_reads = train_set.shape[0] + 1000000
smallest_len = len(train_seq)
rand_reads = np.random.choice(range(smallest_len-152), size = num_reads, replace=False)
myfile = open(root_dir+'text.txt', 'w')
for idx, read_start in tqdm(enumerate(rand_reads), total=num_reads):
    read = str(train_seq[read_start:read_start+read_length]) + '\n'
    myfile.write(read)
myfile.close()

############################################################################################
with open(root_dir+'text.txt') as f:
    content = f.readlines()
content = [x.strip().lower() for x in content] 

############################################################################################
train_set = np.empty([len(content), read_length, 5, 1], np.int8)
for idx, contents in tqdm(enumerate(content), total=len(content)):
    train_set[idx] = transform(np.expand_dims((np.array(list(contents))), axis=0))
    
np.save(root_dir+'chr1_ref_reads', train_set) #already shuffled

100%|██████████| 2685000/2685000 [03:14<00:00, 13819.38it/s]


### Get test reads from viral sequence

In [10]:
viral_seq = 0
for idx, seq in enumerate(SeqIO.parse(root_dir+"agpv1.fa", "fasta")):
    viral_seq = seq.seq
    
train_seq = viral_seq
num_reads = 10000
smallest_len = len(train_seq)
rand_reads = np.random.choice(range(smallest_len-152), size = num_reads, replace=False)
myfile = open('text.txt', 'w')
for idx, read_start in tqdm(enumerate(rand_reads), total=num_reads):
    read = str(train_seq[read_start:read_start+read_length]) + '\n'
    myfile.write(read)
myfile.close()
with open('text.txt') as f:
    content = f.readlines()
content = [x.strip().lower() for x in content] 
train_set = np.empty([len(content), read_length, 5, 1], np.int8)
for idx, contents in tqdm(enumerate(content), total=len(content)):
    train_set[idx] = transform(np.expand_dims((np.array(list(contents))), axis=0))

############################################################################################
np.save('./data/v_ref_reads_test', train_set) #already shuffled

10000


### Get test reads from viral sequence (using nam's reads)

In [48]:
viral_seq = []
for idx, seq in enumerate(SeqIO.parse(root_dir+"run_1.150.0.000000.fas", "fasta")):
    if(seq.id[0]=='v'):
        viral_seq.append(str(seq.seq))
        if(not viral_seq[-1].islower()):
            viral_seq[-1] = viral_seq[-1].lower()

train_set = np.empty([len(viral_seq), read_length, 5, 1], np.int8)
for idx, contents in tqdm(enumerate(viral_seq), total=len(viral_seq)):
    train_set[idx] = transform(np.expand_dims((np.array(list(contents))), axis=0))

############################################################################################
np.save(root_dir+'v_ref_reads_test', train_set) #already shuffled

100%|██████████| 10000/10000 [00:00<00:00, 13590.70it/s]


In [50]:
# print(contents, train_set[-1]) #check

### Create test and training set

#### 1 = virus, 0 human

In [10]:
v_ref_reads = np.load(root_dir + 'v_ref_reads.npy')
np.random.shuffle(v_ref_reads)
h_ref_reads = np.load(root_dir + 'chr1_ref_reads.npy')

hum_train = h_ref_reads[:v_ref_reads.shape[0]]
X_train = np.array([*v_ref_reads, *hum_train])
Y_train = np.array([*np.ones(v_ref_reads.shape[0]), *np.zeros(hum_train.shape[0])])
np.save(root_dir+'train_set.npy', X_train)
np.save(root_dir+'train_set_label.npy', Y_train)

hum_test = h_ref_reads[v_ref_reads.shape[0]:]
viral_test = np.load(root_dir + 'v_ref_reads_test.npy')
X_test = np.concatenate((viral_test, hum_test), axis=0) # 1 = virus, 0 human
Y_test = np.array([*np.ones(viral_test.shape[0]), *np.zeros(hum_test.shape[0])])
np.save(root_dir+'test_set.npy', X_test)
np.save(root_dir+'test_set_label.npy', Y_test)

print("X_train shape: ", X_train.shape, "Y_train shape: ", Y_train.shape, "X_test.shape", X_test.shape)

X_train shape:  (3370000, 150, 5, 1) Y_train shape:  (3370000,) X_test.shape (1010000, 150, 5, 1)


### Generate training set for integrated sequences

In [53]:
num_vreads = v_ref_reads.shape[0]
num_hreads = hum_train.shape[0]
X2_train = np.empty([1000000, read_length, 5, 1], dtype=np.dtype(bool))
Y2_train = np.empty([1000000,read_length], dtype=np.dtype(bool))
num_train = 1000000
pos = np.random.choice(range(read_length-1), size= num_train)
vreads = np.random.choice(range(num_vreads), size= num_train, replace=False)
hreads = np.random.choice(range(num_hreads), size= num_train, replace=False)
p_hstart = np.random.randint(low = 0, high = 2, size=num_train) #probability that the read starts with human seq
for i in range(num_train):
    vread = v_ref_reads[vreads[i]]
    hread = hum_train[hreads[i]]
    GT = np.zeros(150)
    if(p_hstart[i]):
        X2_train[i] = np.vstack((hread[:pos[i]], vread[pos[i]:]))
        GT[pos[i]:] = 1
    else:
        X2_train[i] = np.vstack((vread[:pos[i]], hread[pos[i]:]))
        GT[:pos[i]] = 1
    Y2_train[i] = GT

In [54]:
np.save(root_dir+'integ_train_set.npy', X2_train)
np.save(root_dir+'integ_train_set_label.npy', Y2_train)

In [ ]:
print(vread[pos[i]], ' --', hread[pos[i]-2:pos[i]], ' --', X2_train[i][pos[i]-2:pos[i]+1])

### Generate test set for integrated sequences

In [56]:
num_vreads = viral_test.shape[0]
num_hreads = hum_test.shape[0]
X2_test = np.empty([1000000, read_length, 5, 1], dtype=np.dtype(bool))
Y2_test = np.empty([1000000,read_length], dtype=np.dtype(bool))
num_test = 1000000
pos = np.random.choice(range(read_length-1), size= num_train)
vreads = np.random.choice(range(num_vreads), size= num_test)
hreads = np.random.choice(range(num_hreads), size= num_test)
p_hstart = np.random.randint(low = 0, high = 2, size=num_train) #probability that the read starts with human seq
for i in range(num_train):
    vread = viral_test[vreads[i]]
    hread = hum_test[hreads[i]]
    GT = np.zeros(150)
    if(p_hstart[i]):
        X2_test[i] = np.vstack((hread[:pos[i]], vread[pos[i]:]))
        GT[pos[i]:] = 1
    else:
        X2_test[i] = np.vstack((vread[:pos[i]], hread[pos[i]:]))
        GT[:pos[i]] = 1
    Y2_test[i] = GT

In [57]:
np.save(root_dir+'integ_test_set.npy', X2_test)
np.save(root_dir+'integ_test_set_label.npy', Y2_test)

### Generate reads from all chromosomes

In [ ]:
num_reads = 1000000

for idx, seq in enumerate(SeqIO.parse(root_dir+"hg19full.fa", "fasta")):
    train_seq = seq.seq
    name = seq.id
    seq_len = len(train_seq)
    
    if(len(train_seq) < num_reads):
        num_reads = seq_len-151
        
    rand_reads = np.random.choice(range(num_reads), size = num_reads, replace=False)
    myfile = open(root_dir+name+'.txt', 'w')
    for idx, read_start in tqdm(enumerate(rand_reads), total=num_reads):
        read = str(train_seq[read_start:read_start+read_length]) + '\n'
        myfile.write(read)
    myfile.close()
    with open(root_dir+name+'.txt') as f:
        content = f.readlines()
    content = [x.strip().lower() for x in content] 

    train_set = np.empty([len(content), read_length, 5, 1], np.int8)
    for idx, contents in tqdm(enumerate(content), total=len(content)):
        train_set[idx] = transform(np.expand_dims((np.array(list(contents))), axis=0))
    
    np.save(root_dir+name, train_set) #already shuffled

 37%|███▋      | 372340/1000000 [00:26<00:44, 14055.74it/s]

In [20]:
import glob
all_chr = np.empty([1, 150, 5, 1], np.bool)
for file in glob.glob(root_dir+"/chr_reads/"+"*.npy"):
    temp = np.load(file)
    all_chr = np.vstack((all_chr, temp))

In [21]:
all_chr.shape

(24016421, 150, 5, 1)